# ST7735 Display Project Documentation

## Development Notes

### Upload Issues (November 24, 2025)

**Critical Requirement:** Always disconnect serial monitor before uploading firmware

- Upload fails with "SAM-BA operation failed" when serial monitor is connected
- The serial monitor holds the programming port open, preventing firmware upload
- This applies to both VS Code serial monitor and external terminal applications

### Serial Communication Configuration

**Baud Rate Discovery:**
- Found firmware was incorrectly set to 2,000,000 baud (intended for bitmap transfers only)
- Standard debug output should use **115200 baud**
- Other project components (cal_lcd.cpp, documentation) consistently use 115200

**Action Taken:**
- Updated `src/main.cpp` line 43: `SerialUSB.begin(115200);`
- This aligns with calibration tool and other project documentation

**Pre-Upload Checklist:**
1. Disconnect serial monitor
2. Verify baud rate is appropriate for the operation (115200 for debug, 2M for bitmap transfers)
3. Use Native USB port (/dev/ttyACM0) for communication
4. Programming port is for upload only

## Diagonal Line Bug Investigation (November 18-24, 2025)

### Bug Description
- Ragged diagonal line from [0,0] to center on LCD calibration display
- Gaps appear in different locations between portrait and landscape modes
- Line should be smooth and continuous

### Root Cause Analysis
1. **Center Calculation Issue:** Original code used `config.width/height` instead of runtime `tft->width()/height()`
2. **Line Drawing Algorithm:** Standard `drawLine()` function may have fractional pixel gaps
3. **Color Configuration:** Code was using `ST77XX_YELLOW` but test changed to bright white (`0xFFFF`)

### Implementation Status
- **Test Implementation:** Manual Bresenham algorithm added to `lib/DisplayManager/DisplayManager.cpp`
- **Code Location:** Lines 91-127 in `drawCalibrationFrame()` function
- **Debug Output:** Added SerialUSB.println() statements for execution tracing
- **Current Issue:** Debug output not appearing - need to verify execution path

### Next Steps
1. Upload corrected firmware with 115200 baud rate
2. Connect serial monitor to verify debug output appears
3. Confirm Bresenham algorithm is executing
4. Evaluate if manual pixel drawing eliminates the ragged appearance

### Test Mode Changes (November 24, 2025)

**Gradient Background Disabled:**
- **Location:** `lib/DisplayManager/DisplayManager.cpp` line ~65 in `showTestPattern()`
- **Reason:** Gradient was being overwritten by black background anyway, causing unnecessary overhead
- **Status:** Permanently disabled during diagonal line testing
- **Impact:** Test patterns now show calibration elements on pure black background

**Debug Output Confirmed Working:**
- Serial communication established on `/dev/ttyACM1` at 115200 baud
- Bresenham algorithm executing successfully on both displays:
  - Display 1 (160x128): Center at [80,64]  
  - Display 2 (128x160): Center at [64,80]
- All debug trace messages appearing correctly

**Current Test Status:**
- Manual Bresenham line drawing algorithm implemented
- Bright white diagonal (0xFFFF) confirmed executing
- Background gradient interference eliminated
- Ready for diagonal line quality assessment